In [ ]:
import ollama
import gradio as gr

In [ ]:
MODEL = 'deepseek-v3.1:671b-cloud'
system_message = """
You are a helpful assistant for an Airline called FlightAI.
Give short, courteous answers, no more than 1 sentence.
Always be accurate. If you don't know the answer, say so.
"""

## Tools

Tools are an incredibly powerful feature provided by the frontier LLMs.

With tools, you can write a function, and have the LLM call that function as part of its response.

Sounds almost spooky.. we're giving it the power to run code on our machine?

Well, kinda.

In [ ]:
# Let's start by making a useful function

ticket_prices = {'london': '$799', 'paris': '$899', 'tokyo':'$1400', 'berlin': '$499'}
age_group_discounts = {'baby': '90%', 'children': '50%', 'elders':'30%'}

def get_ticket_price(destination_city):
    print(f'Tool get_ticket_price called for {destination_city}')
    city = destination_city.lower()
    return ticket_prices.get(city, 'Unknown')

def get_discount(age_group):
    print(f'Tool get_discount called for {age_group}')
    age_group = age_group.lower()
    return age_group_discounts.get(age_group, 'Unknown')

In [ ]:
# There's a particular dictionary structure that's required to describe our function:

price_function = {
    'type': 'function',
    'function': {
        "name": "get_ticket_price",
        "description": "Get the price of a return ticket to the destination city.",
        "parameters": {
            "type": "object",
            "properties": {
                "destination_city": {
                    "type": "string",
                    "description": "The city that the customer wants to travel to",
                },
            },
            "required": ["destination_city"],
            "additionalProperties": False
        }
    }
}

discount_function = {
    'type': 'function',
    'function': {
        "name": "get_discount",
        "description": "The percentage of discount for people of certain age group.",
        "parameters": {
            "type": "object",
            "properties": {
                "age_group": {
                    "type": "string",
                    "description": "Approximate age group that the traveler falls into (baby, children, elders)",
                },
            },
            "required": ["age_group"],
            "additionalProperties": False
        }
    }
}

In [ ]:
# And this is included in a list of tools:

tools = [ price_function, discount_function ]
available_functions = {
    'get_ticket_price': get_ticket_price,
    'get_discount': get_discount
}

## Getting Ollama to use our Tool

What we actually do is give the LLM the opportunity to inform us that it wants us to run the tool.

Here's how the new chat function looks:

In [ ]:
def chat(message, history):
    messages = [{'role':'system', 'content': system_message}]
    for user_message, assistant_message in history:
        messages.append({'role':'user', 'content': user_message})
        messages.append({'role':'assistant', 'content': assistant_message})
    messages.append({'role':'user', 'content': message})
    
    response = ollama.chat(model=MODEL, messages=messages, tools=tools)
    
    # using tools for ollama

    if response.message.tool_calls:
        # collect all tool outputs here
        tool_results = []  

        # There may be multiple tool calls in the response
        for tool in response.message.tool_calls:
            # Ensure the function is available, and then call it
            if function_to_call := available_functions.get(tool.function.name):
                print('Calling function:', tool.function.name)
                print('Arguments:', tool.function.arguments)
                output = function_to_call(**tool.function.arguments)
                print('Function output:', output)

                # store each tool result properly
                tool_results.append({
                    "role": "tool",
                    "content": str(output),
                    "tool_name": tool.function.name,
                })
            else:
                print('Function', tool.function.name, 'not found')

        # Add the function response to messages for the model to use
        messages.append(response.message)
        # messages.append({'role': 'tool', 'content': str(output), 'tool_name': tool.function.name})
        messages.extend(tool_results)

        # Get final response from model with function outputs
        final_response = ollama.chat(model=MODEL, messages=messages)
    
        return final_response['message']['content']
    
    return response['message']['content']

In [ ]:
gr.ChatInterface(fn = chat).launch()